In [1]:
import sys
sys.path.append("../../pkgs")
import pandas as pd
import numpy as np
from sksurv.metrics import concordance_index_censored,concordance_index_ipcw
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sksurv.datasets import get_x_y
from sksurv.linear_model import CoxPHSurvivalAnalysis, CoxnetSurvivalAnalysis
import joblib
import pickle

# from util.setting import DEFAULT_COL_DTYPE, DEFAULT_LABELS, DEFAULT_CONT_FEATURES, DEFAULT_FEATURES, BEST_EXTEND_FEATURES
# from util.data import standard_transform
# from util.eval import risk_cindex, eval_cindex

In [2]:
# clinical_model = joblib.load('./result/OP_post_2year_coxnet_model_advance.joblib')
clinical_model = joblib.load('./result/OP_post_overall_coxnet_model_advance.joblib')
clinical_features = clinical_model.feature_names_in_
print(clinical_features)

['Tumor size' 'lnAST' 'BCLC' 'Tumor number' 'ALBIgrade'
 'class_Histologic grade' 'class_AFP' 'Ishak' 'Steatosis grade' 'K'
 'class_Steatosis grade' 'MVI' 'BMI' 'AST' 'HBsAg']


In [3]:
train = pd.read_csv('./data/post_overall/HCC OP 20230511 train_post_input_derived features.csv')
test = pd.read_csv('./data/post_overall/HCC OP 20230511 test_post_input_derived features.csv')
train_std = pd.read_csv('./data/post_overall/HCC OP 20230511 train_post_input_derived features_std.csv')
test_std = pd.read_csv('./data/post_overall/HCC OP 20230511 test_post_input_derived features_std.csv')
print(len(train),len(test),len(train_std),len(test_std))

1033 444 1033 444


In [38]:
selected = ['Tumor size', 'ln_AST' , 'BCLC','Tumor number', 'ALBIgrade',
 'class_Histologic grade', 'class_AFP', 'Ishak','Steatosis grade', 'K' ,'class_Steatosis grade','MVI','BMI'
 ,'AST','HBsAg',]

In [39]:
train_X = train[ selected ]
train_y = train_std[['Recur', 'Recurrence free survival']].to_records(index=False).astype([('Status', 'bool'), ('Survival', 'float64')])

In [40]:
model = Pipeline([('scaler', StandardScaler()), ('cox', CoxnetSurvivalAnalysis(alphas=clinical_model.alphas, l1_ratio=clinical_model.l1_ratio, fit_baseline_model=True))])

In [41]:
print(len(train_X),len(train_y))

1033 1033


In [42]:
model.fit(train_X, train_y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('cox', CoxnetSurvivalAnalysis(fit_baseline_model=True))])

In [20]:
ibcgas_cindex = eval_cindex(model, train_X[ibcga_features], train_y, test_X[ibcga_features], test_y)

NameError: name 'eval_cindex' is not defined

In [43]:
preds_test = model.predict(test[ selected ])
concordance_index_censored(test_std['Recur'] > 0, test_std['Recurrence free survival'],preds_test)

(0.7026961587215679, 49337, 20874, 0, 7)

In [44]:
joblib.dump(model, "./result/OP_post_overall_coxnet_model_advance_piplines.bin")

['./result/OP_post_overall_coxnet_model_advance_piplines.bin']

In [45]:
from sklearn.metrics import roc_auc_score

# 假設您已經建立好並適配了 Cox 模型，並命名為 `cox_model`
# 請確保您的模型已經適配了訓練數據

# 使用 Cox 模型預測風險分數
predicted_scores = model.predict(test_std[selected])

# 計算 AUC
auc = roc_auc_score(test_std['Recur'], predicted_scores)

print("AUC:", auc)

AUC: 0.6449555628703094
